In [3]:
from __future__ import division

import sys
import time
import logging
import numpy as np
from scipy.io import loadmat
from os import path

# change this to the path of mushu if you don't have it in your
# PYTHONPATH already
sys.path.append('../../mushu')
sys.path.append('../')

import libmushu
from libmushu.driver.replayamp import ReplayAmp
from wyrm.types import RingBuffer
import wyrm.processing as proc
from wyrm import io
from wyrm.types import Data, BlockBuffer, RingBuffer
from pandas import read_csv      

#Classifiers
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn import metrics
import matplotlib
import matplotlib.pyplot as plt

from sklearn.externals import joblib
from os import listdir
from os.path import isfile, join

import pickle

In [36]:
STIMULUS_CODE = {
    # cols from left to right
    1 : "agmsy5",
    2 : "bhntz6",
    3 : "ciou17",
    4 : "djpv28",
    5 : "ekqw39",
    6 : "flrx4_",
    # rows from top to bottom
    7 : "abcdef",
    8 : "ghijkl",
    9 : "mnopqr",
    10: "stuvwx",
    11: "yz1234",
    12: "56789_"
}

MARKER_DEF_TRAIN = {'target': ['target'], 'nontarget': ['nontarget']}
MARKER_DEF_TEST = {i : [i] for i in STIMULUS_CODE.values()}

SEG_IVAL = [0, 800]

In [44]:
def train(dat):
    cnt = dat

    fs_n = cnt.fs / 2

    b, a = proc.signal.butter(5, [10 / fs_n], btype='low')
    cnt = proc.lfilter(cnt, b, a)

#     b, a = proc.signal.butter(5, [.4 / fs_n], btype='high')
#     cnt = proc.lfilter(cnt, b, a)

    cnt = proc.subsample(cnt, 20)

    epo = proc.segment_dat(cnt, MARKER_DEF_TRAIN, SEG_IVAL)

    #from wyrm import plot
    #plot.plot_spatio_temporal_r2_values(proc.sort_channels(epo))
    #print JUMPING_MEANS_IVALS
    #plot.plt.show()

    fv = proc.create_feature_vectors(epo)
    X = fv.data
    y = fv.axes[0]
#     X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size = 0.3, train_size = 0.7)

    #     clf = proc.lda_train(fv)
    clf = LinearDiscriminantAnalysis(solver='eigen')
    clf.fit(X,y)
    return clf

In [31]:
# Getting the data objects
with open('data_subjs.pkl','rb') as f:  
    train_A,train_Abis,train_B,train_Bbis,train_C,test_A,test_Abis,test_B,test_Bbis,test_C = pickle.load(f)

In [51]:
train_data = {'A': train_A, 'Abis': train_Abis, 'B': train_B, 'Bbis': train_Bbis, 'C': train_C}
test_data = {'A': test_A, 'Abis': test_Abis, 'B': test_B, 'Bbis': test_Bbis, 'C': test_C}

In [52]:
for s in train_data.keys():
    subject = s
    clf = train(train_data[subject])
    filename = 'Trained Classifiers/'+subject+'/LDA_'+subject+'.pkl'
    joblib.dump(clf, filename)